# 💗 [Day 9](https://adventofcode.com/2021/day/9)

In [1]:
import numpy as np

def conv2d(flt, arr, padding=0):
    out = np.zeros_like(arr)
    left = int(np.floor(flt.shape[0] / 2))
    right = int(np.ceil(flt.shape[0] / 2))
    top = int(np.floor(flt.shape[1] / 2))
    bottom = int(np.ceil(flt.shape[1] / 2))
    arr = np.pad(arr, ((left, right), (top, bottom)), 
                 mode='constant', constant_values=padding)
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            out[i, j] = np.sum(flt * arr[i:i + left + right,
                                         j:j + top + bottom])
    return out


def get_low_points(arr):
    filters = np.zeros((4, 3, 3))
    filters[:, 1, 1] = -1
    filters[0, 0, 1] = 1  # top
    filters[1, 1, 2] = 1  # right
    filters[2, 2, 1] = 1  # bottom
    filters[3, 1, 0] = 1  # left
    is_low_point = sum(conv2d(filters[i], arr, padding=9) <= 0 for i in range(4)) == 0
    risk_levels = np.sum(is_low_point * (arr + 1))
    return is_low_point, risk_levels


def expand_bassin(arr, start):
    queue = [start]
    bassin_size = 0
    while len(queue):
        x, y = queue.pop()
        if 0 <= x < arr.shape[0] and 0 <= y < arr.shape[1]:
            if 9 > arr[x, y] >= 0:
                bassin_size += 1
                queue.append((x - 1, y))
                queue.append((x + 1, y))
                queue.append((x, y - 1))
                queue.append((x, y + 1))
                arr[x, y] = 9
    return bassin_size
    

def get_bassins(arr):
    # The low points are the center of bassins (because if a point 
    # is not a low point, then it can always flow in to something 
    # else). So we can simply expand the low points
    low_points, _ = get_low_points(arr)
    bassin_sizes = [expand_bassin(arr, start) for start in np.argwhere(low_points)]
    bassin_sizes = sorted(bassin_sizes, key= lambda x:- x)
    return bassin_sizes[0] * bassin_sizes[1] * bassin_sizes[2]

In [2]:
%%time
with open('inputs/day09.txt', 'r') as f:
    inputs = f.read()
    inputs = np.array([[int(c) for c in x] for x in inputs.splitlines()])
    
    
print(f"The risk level of the low points is \033[92;1m{get_low_points(inputs)[1]}\033[0m")
print(f"The top-3 largest bassins sizes multiplied together yield \033[92;1m{get_bassins(inputs)}\033[0m")
print()

The risk level of the low points is 548
The top-3 largest bassins sizes multiplied together yield 786048

CPU times: user 946 ms, sys: 3.4 ms, total: 949 ms
Wall time: 955 ms
